In [1]:
# This is attention based residual network architecture design
from __future__ import print_function
import keras
from tensorflow.keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from tensorflow.keras.layers import AveragePooling2D,GlobalAveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from tensorflow.keras.layers import AveragePooling2D, Input, Flatten, Add
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model
from tensorflow.keras.initializers import glorot_uniform
from datetime import datetime

# attention modules importing
import attention_modules
from attention_modules import attach_attention_module


Using TensorFlow backend.


Instructions for updating:
Use `tf.config.run_functions_eagerly` instead of the experimental version.


In [2]:
def identity_block(X, f, filters, stage, block, attention_module=None):
    """
    Implementation of the identity block for a RseNet model
    
    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    stage -- integer, used to name the layers, depending on their position in the network
    block -- string/character, used to name the layers, depending on their position in the network
    
    Returns:
    X -- output of the identity block, tensor of shape (n_H, n_W, n_C)
    """
    
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value. You'll need this later to add back to the main path. 
    X_shortcut = X
    
    # First component of main path
    X = Conv2D(filters = F1, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    
    # Second component of main path (≈3 lines)
    X = Conv2D(filters = F2, kernel_size = (f, f), strides = (1,1), padding = 'same', name = conv_name_base + '2b', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    # Third component of main path (≈2 lines)
    X = Conv2D(filters = F3, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2c', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)

    # attention module
    #if attention_module is not None:
    #    Y = attach_attention_module(X, attention_module)
    #X = Add()([X_shortcut, Y])
    
    # Final step: Add shortcut value to main path, and pass it through a RELU activation (≈2 lines)
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    
    
    return X

In [3]:
def convolutional_block(X, f, filters, stage, block, s = 2, attention_module=None):
    """
    Implementation of the convolutional block as defined in Figure 4
    
    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    stage -- integer, used to name the layers, depending on their position in the network
    block -- string/character, used to name the layers, depending on their position in the network
    s -- Integer, specifying the stride to be used
    
    Returns:
    X -- output of the convolutional block, tensor of shape (n_H, n_W, n_C)
    """
    
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value
    X_shortcut = X


    ##### MAIN PATH #####
    # First component of main path 
    X = Conv2D(F1, (1, 1), strides = (s,s), name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    # Second component of main path (≈3 lines)
    X = Conv2D(filters = F2, kernel_size = (f, f), strides = (1,1), padding = 'same', name = conv_name_base + '2b', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
    X = Activation('relu')(X)


    # Third component of main path (≈2 lines)
    X = Conv2D(filters = F3, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2c', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)


    ##### SHORTCUT PATH #### (≈2 lines)
    X_shortcut = Conv2D(filters = F3, kernel_size = (1, 1), strides = (s,s), padding = 'valid', name = conv_name_base + '1',
                        kernel_initializer = glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis = 3, name = bn_name_base + '1')(X_shortcut)

    # attention modules added
    if attention_module is not None:
        X = attach_attention_module(X, attention_module)
    X = Add()([X, X_shortcut])
    
    # Final step: Add shortcut value to main path, and pass it through a RELU activation (≈2 lines)
    #X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    
    
    return X

In [4]:
def ResNet20(input_shape=(224, 224, 3), classes=11, attention_module=None):
    """
    Implementation of the custom ResNet20 the following architecture:
    CONV2D -> BATCHNORM -> RELU -> MAXPOOL -> CONVBLOCK -> CONVBLOCK -> IDBLOCK*1
    -> CONVBLOCK -> IDBLOCK*1 -> CONVBLOCK -> AVGPOOL -> TOPLAYER

    Arguments:
    input_shape -- shape of the images of the dataset
    classes -- integer, number of classes

    Returns:
    model -- a Model() instance in Keras
    """

    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)

    # Zero-Padding
    X = ZeroPadding2D((3, 3))(X_input)

    # Stage 1
    X = Conv2D(64, (7, 7), strides=(2, 2), name='conv1', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name='bn_conv1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    # Stage 2
    X = convolutional_block(X, f=3, filters=[64, 64, 256], stage=2, block='a', s=1)
    #X = identity_block(X, 3, [64, 64, 256], stage=2, block='b')
    #X = identity_block(X, 3, [64, 64, 256], stage=2, block='c')

    ### START CODE HERE ###

    # Stage 3 (≈4 lines)
    X = convolutional_block(X, f = 3, filters = [128, 128, 512], stage = 3, block='a', s = 2)
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='b')
    #X = identity_block(X, 3, [128, 128, 512], stage=3, block='c')
    #X = identity_block(X, 3, [128, 128, 512], stage=3, block='d')
    
   
    # Stage 4 (≈6 lines)
    X = convolutional_block(X, f = 3, filters = [256, 256, 1024], stage = 4, block='a', s = 2, attention_module=attention_module)
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='b')
    #X = identity_block(X, 3, [256, 256, 1024], stage=4, block='c')
    #X = identity_block(X, 3, [256, 256, 1024], stage=4, block='d')
    #X = identity_block(X, 3, [256, 256, 1024], stage=4, block='e')
    #X = identity_block(X, 3, [256, 256, 1024], stage=4, block='f')
    
    # attention module
    #if attention_module is not None:
    #    Y = attach_attention_module(X, attention_module)
    #X = Add()([X, Y])

    # Stage 5 (≈3 lines)
    X = convolutional_block(X, f = 3, filters = [512, 512, 2048], stage = 5, block='a', s = 2)
    #X = identity_block(X, 3, [512, 512, 2048], stage=5, block='b')
    #X = identity_block(X, 3, [512, 512, 2048], stage=5, block='c')

    # AVGPOOL (≈1 line). Use "X = AveragePooling2D(...)(X)"
    #X = AveragePooling2D((2,2), name="avg_pool")(X)
    X = GlobalAveragePooling2D(name="avg_pool")(X)

    ### END CODE HERE ###

    # output layer
    #X = Flatten()(X)
    X = Dense(classes, activation='softmax', name='fc_' + str(classes), kernel_initializer = glorot_uniform(seed=0))(X)
    
    # for naming the model
    if attention_module is not None:
        attention_name = '_'+attention_module.split('_')[0]
    else:
        attention_name = ''
    # Create model
    model = Model(inputs = X_input, outputs = X, name='ResNet20'+attention_name)

    return model

In [16]:
def LW_ResNet20(input_shape=(224, 224, 3), classes=11, attention_module=None):
    """
    Implementation of the custom ResNet20 the following architecture:
    CONV2D -> BATCHNORM -> RELU -> MAXPOOL -> CONVBLOCK -> CONVBLOCK -> IDBLOCK*1
    -> CONVBLOCK -> IDBLOCK*1 -> CONVBLOCK -> AVGPOOL -> TOPLAYER

    Arguments:
    input_shape -- shape of the images of the dataset
    classes -- integer, number of classes

    Returns:
    model -- a Model() instance in Keras
    """

    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)

    # Zero-Padding
    X = ZeroPadding2D((3, 3))(X_input)

    # Stage 1
    X = Conv2D(16, (7, 7), strides=(2, 2), name='conv1', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name='bn_conv1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    # Stage 2
    X = convolutional_block(X, f=3, filters=[16, 16, 64], stage=2, block='a', s=1)
   
    ### START CODE HERE ###

    # Stage 3 (≈4 lines)
    X = convolutional_block(X, f = 3, filters = [32, 32, 128], stage = 3, block='a', s = 2)
    X = identity_block(X, 3, [32, 32, 128], stage=3, block='b')
       
    # Stage 4 (≈6 lines)
    X = convolutional_block(X, f = 3, filters = [64, 64, 256], stage = 4, block='a', s = 2, attention_module=attention_module)
    X = identity_block(X, 3, [64, 64, 256], stage=4, block='b')
   
    # Stage 5 (≈3 lines)
    X = convolutional_block(X, f = 3, filters = [256, 256, 1024], stage = 5, block='a', s = 2)
    #X = identity_block(X, 3, [512, 512, 2048], stage=5, block='b')
    #X = identity_block(X, 3, [512, 512, 2048], stage=5, block='c')

    # AVGPOOL (≈1 line). Use "X = AveragePooling2D(...)(X)"
    #X = AveragePooling2D((2,2), name="avg_pool")(X)
    X = GlobalAveragePooling2D(name="avg_pool")(X)

    ### END CODE HERE ###

    # output layer
    #X = Flatten()(X)
    X = Dense(classes, activation='softmax', name='fc_' + str(classes), kernel_initializer = glorot_uniform(seed=0))(X)
    
    # for naming the model
    if attention_module is not None:
        attention_name = '_'+attention_module.split('_')[0]
    else:
        attention_name = ''
    # Create model
    model = Model(inputs = X_input, outputs = X, name='LW_ResNet20'+attention_name)

    return model

In [17]:
# model = ResNet20(input_shape = (256, 256, 3), classes = 11, attention_module='ca_block')
model = LW_ResNet20(input_shape = (256, 256, 3), classes = 11, attention_module=None)
model.summary()

Model: "LW_ResNet20"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
zero_padding2d_3 (ZeroPadding2D (None, 262, 262, 3)  0           input_4[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 128, 128, 16) 2368        zero_padding2d_3[0][0]           
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 128, 128, 16) 64          conv1[0][0]                      
________________________________________________________________________________________

In [18]:
model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])


In [19]:
# image preprocessing
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   rotation_range=40,
                                   horizontal_flip=True,
                                   vertical_flip=True,
                                   fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255)


batch_size = 32

train_data_dir = "D:/Anil/Dataset/tomato/train"     # directory of training data

test_data_dir = "D:/Anil/Dataset/tomato/val"      # directory of Validation data

training_set = train_datagen.flow_from_directory(train_data_dir,
                                                 target_size=(256, 256),
                                                 batch_size=batch_size,
                                                 class_mode='categorical')

test_set = test_datagen.flow_from_directory(test_data_dir,
                                            target_size=(256, 256),
                                            batch_size=batch_size,
                                            class_mode='categorical')

print(training_set.class_indices)



Found 15156 images belonging to 11 classes.
Found 1891 images belonging to 11 classes.
{'bacterial_spot': 0, 'early_blight': 1, 'fusarium_wilt': 2, 'healthy': 3, 'late_blight': 4, 'leaf_mold': 5, 'mosaic_virus': 6, 'septoria_leaf_spot': 7, 'spider_mites': 8, 'target_spot': 9, 'yellow_leaf_curl_virus': 10}


In [20]:
# checkpoint
#weightpath = "D:/Anil/saved_model/SACNN/ResNet50_tomato_adam_100_epochs.hdf5"
weightpath = "CBAM-keras/saved_models/LW_ResNet20_tomato_adam_100_epochs.hdf5"
checkpointer = ModelCheckpoint(weightpath, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')

epochs = 100

start_time = datetime.now()
print("start time: "+str(start_time))

#fitting images to CNN
history = model.fit(training_set,
                         steps_per_epoch=training_set.samples//batch_size,
                         validation_data=test_set,
                         epochs=epochs,
                         validation_steps=test_set.samples//batch_size,
                         callbacks=[checkpointer])#callbacks_list)


# #fitting images to CNN
# history = classifier.fit_generator(training_set,
#                                    steps_per_epoch=training_set.samples//batch_size,
#                                    validation_data=test_set,
#                                    epochs=10,
#                                    validation_steps=test_set.samples//batch_size)

# Stop time
stop_time = datetime.now()
print("stop time: "+str(stop_time))

total_time = stop_time - start_time
print("Total time: "+str(total_time))


start time: 2021-12-24 14:31:12.087475
Epoch 1/100
473/473 [==============================] - 135s 285ms/step - loss: 1.0875 - accuracy: 0.6479 - val_loss: 4.5378 - val_accuracy: 0.5143

Epoch 00001: val_loss improved from inf to 4.53780, saving model to CBAM-keras/saved_models\LW_ResNet20_tomato_adam_100_epochs.hdf5
Epoch 2/100
473/473 [==============================] - 135s 285ms/step - loss: 0.5442 - accuracy: 0.8138 - val_loss: 7.6877 - val_accuracy: 0.3008

Epoch 00002: val_loss did not improve from 4.53780
Epoch 3/100
473/473 [==============================] - 136s 287ms/step - loss: 0.4204 - accuracy: 0.8552 - val_loss: 3.5853 - val_accuracy: 0.5387

Epoch 00003: val_loss improved from 4.53780 to 3.58526, saving model to CBAM-keras/saved_models\LW_ResNet20_tomato_adam_100_epochs.hdf5
Epoch 4/100
473/473 [==============================] - 135s 285ms/step - loss: 0.3691 - accuracy: 0.8737 - val_loss: 2.8194 - val_accuracy: 0.5143

Epoch 00004: val_loss improved from 3.58526 to 2.8


Epoch 00037: val_loss did not improve from 0.07157
Epoch 38/100
473/473 [==============================] - 135s 284ms/step - loss: 0.0597 - accuracy: 0.9781 - val_loss: 0.1138 - val_accuracy: 0.9650

Epoch 00038: val_loss did not improve from 0.07157
Epoch 39/100
473/473 [==============================] - 134s 283ms/step - loss: 0.0540 - accuracy: 0.9820 - val_loss: 0.1557 - val_accuracy: 0.9439

Epoch 00039: val_loss did not improve from 0.07157
Epoch 40/100
473/473 [==============================] - 134s 284ms/step - loss: 0.0534 - accuracy: 0.9812 - val_loss: 0.1840 - val_accuracy: 0.9407

Epoch 00040: val_loss did not improve from 0.07157
Epoch 41/100
473/473 [==============================] - 134s 284ms/step - loss: 0.0741 - accuracy: 0.9763 - val_loss: 0.4427 - val_accuracy: 0.8972

Epoch 00041: val_loss did not improve from 0.07157
Epoch 42/100
473/473 [==============================] - 134s 284ms/step - loss: 0.0513 - accuracy: 0.9825 - val_loss: 0.0601 - val_accuracy: 0.9820


473/473 [==============================] - 135s 286ms/step - loss: 0.0303 - accuracy: 0.9903 - val_loss: 0.5713 - val_accuracy: 0.9110

Epoch 00077: val_loss did not improve from 0.01940
Epoch 78/100
473/473 [==============================] - 135s 286ms/step - loss: 0.0297 - accuracy: 0.9901 - val_loss: 0.9199 - val_accuracy: 0.8522

Epoch 00078: val_loss did not improve from 0.01940
Epoch 79/100
473/473 [==============================] - 136s 286ms/step - loss: 0.0362 - accuracy: 0.9890 - val_loss: 0.0511 - val_accuracy: 0.9841

Epoch 00079: val_loss did not improve from 0.01940
Epoch 80/100
473/473 [==============================] - 135s 286ms/step - loss: 0.0290 - accuracy: 0.9907 - val_loss: 0.8955 - val_accuracy: 0.8575

Epoch 00080: val_loss did not improve from 0.01940
Epoch 81/100
473/473 [==============================] - 134s 284ms/step - loss: 0.0200 - accuracy: 0.9932 - val_loss: 0.0218 - val_accuracy: 0.9926

Epoch 00081: val_loss did not improve from 0.01940
Epoch 82/100


In [21]:
# another way to save it.
import pandas as pd
hist_df = pd.DataFrame(history.history) 
#hist_csv_file = "CBAM-keras/saved_models/ResNet20_ca_block_tomato_adam_100_epochs.csv"
hist_csv_file = "CBAM-keras/saved_models/LW_ResNet20_tomato_adam_100_epochs.csv"
# hist_csv_file = 'history.csv'
with open(hist_csv_file, mode='w', newline='') as f:
    hist_df.to_csv(f)


In [ ]:
# testing of model
import os
test_data_dir = 'D:/Anil/Dataset/tomato/test_imgs'
batch_size = 32
img_width, img_height = 256, 256


In [ ]:
def gen_image_label(directory):
    ''' A generator that yields (label, id, jpg_filename) tuple.'''
    for root, dirs, files in os.walk(directory):
        for f in files:
            _, ext = os.path.splitext(f)
            if ext != '.jpg':
                continue
            basename = os.path.basename(f)
            splits = basename.split('.')
            if len(splits) == 3:
                label, id_, ext = splits
            else:
                label = None
                id_, ext = splits
            fullname = os.path.join(root, f)
            yield label, int(id_), fullname
            

In [ ]:
# Wrap testing data into pandas' DataFrame.
lst = list(gen_image_label(test_data_dir))
test_df = pd.DataFrame(lst, columns=['label', 'id', 'filename'])
test_df = test_df.sort_values(by=['label', 'id'])
test_df['label_code'] = test_df.label.map({'bacterial_spot':0,'early_blight':1,'fusarium_wilt':2,'healthy':3, 'late_blight':4, 'leaf_mold':5,
                                           'mosaic_virus':6, 'septoria_leaf_spot':7, 'spider_mites':8, 'target_spot':9, 
                                           'yellow_leaf_curl_virus':10})

test_df.head(100)

In [ ]:
Y_true = test_df.label_code
print(Y_true)

In [ ]:
# No need to run again once the file save in computer
import numpy as np
from tensorflow.keras.preprocessing import image
img_width = 256
img_height = 256
images = []
for img in test_df.filename:
    img = image.load_img(img, target_size=(img_width, img_height))
    img = image.img_to_array(img)
    img = img/255.0
    img = np.expand_dims(img, axis=0)
    images.append(img)
# print(images)
# np.save("D:/Anil/Dataset/ageng_conf/test_imgs_299x299", images)
# stack up images list to pass for prediction
images = np.vstack(images)


In [ ]:
test_start = datetime.now()
Y_pred = model.predict(images, batch_size=8)
Y_pred_classes = np.argmax(Y_pred, axis = 1)
# classes = model.predict_classes(images, batch_size=8)
print(Y_pred_classes)
test_finish = datetime.now()
test_time = test_finish - test_start
print(test_time)


In [ ]:
from sklearn.metrics import confusion_matrix
from mlxtend.plotting import plot_confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

cls_name = ['bacterial_spot','early_blight','fusarium_wilt','healthy', 'late_blight', 'leaf_mold',
                                           'mosaic_virus', 'septoria_leaf_spot', 'spider_mites', 'target_spot', 
                                           'yellow_leaf_curl_virus']

ytrue = Y_true.values.flatten()
cm = confusion_matrix(ytrue, Y_pred_classes)
fig, ax = plot_confusion_matrix(conf_mat=cm,
                                figsize = (11,9),
                               show_absolute=False,
                               show_normed=True,
                               colorbar=True,
                               class_names = cls_name)
plt.show()
# print(cm)

In [ ]:
print(classification_report(Y_true, Y_pred_classes))
print('test accuracy: ', accuracy_score(Y_true, Y_pred_classes))
